In [ ]:
class threadMain:
    def __init__(self, client , file, engine_azure):
        #set variables
        self.client = client

        #sets variables for the rules
        self.list_regrasGMV = []
        self.list_regrasOther = []

        #set dates
        
        #set engine and connection
        self.engine_azure = engine_azure
        self.conn_azure = self.engine_azure.connect()

        #initialize client subthreads
        self.subthreads = []

        self.file = file
        #start the thread
        self.t = Thread(target=self.threadMainTask, args=())
        self.t.start()

    def getThread(self):
        return (self.t)
    
    def _getaccesstoken(self):
        #get access token from meli
        overrideHeaders = ""
        ending = ""
        extraparams = {
            'grant_type' : 'refresh_token',
            'client_id' : self.client._client_id,
            'client_secret' : self.client._client_secret,
            'refresh_token' : self.client.new_refresh_token
            }
        response = setupAPIrequest(globals()['util']['meli_get_access_token'], extraparams, overrideHeaders, ending)
        #response is in json format
        response_dict = json.loads(response.text)
        #update client class with access token and new info
        self.client.access_token = response_dict['access_token']
        self.client.used_refresh_token = self.client.new_refresh_token
        self.client.new_refresh_token = response_dict['refresh_token']
        self.client.user_id = response_dict['user_id']
    
    def _createsubThread(self):
        """
        creates a subthread for that info
        """
        return 

    def _getordersfromseller(self): #todo -> dividir por thread por página para acelerar os downloads, descobrir porque ele fica vindo pedido repetido embora muda de página
        #seller as same as client
        #get orders from meli
        time_from = (datetime.now() - timedelta(days = 5)).strftime("%Y-%m-%d") + "T00:00:00-00:00"
        time_to = datetime.now().strftime("%Y-%m-%d") + "T" + datetime.now().strftime("%H:%M:%S") + "-00:00"
        print (time_from)
        print (time_to)
        ending = ""
        extraparams = {
            'seller' : self.client.user_id,
            'order.date_created.from' : time_from,
            'order.date_created.to'   : time_to
            }
        #add a header
        overrideHeaders = {
            "Authorization": "Bearer %s" %self.client.access_token
            }
        response = setupAPIrequest(globals()['util']['meli_get_seller_orders'], extraparams, overrideHeaders, ending)
        response_dict = json.loads(response.text)

        
        #page_number
        page_number = response_dict["paging"]["total"]
        for page in range(page_number):
            extraparams['offset'] = page

            t = subthreadMain()
            self.subthreads.append(t.getsubThread())
            
            for t in self.subthreads:
                t.join()

            response = setupAPIrequest(globals()['util']['meli_get_seller_orders'], extraparams, overrideHeaders, ending)
            response_dict = json.loads(response.text)
            #create the orders list of client
            for each_order in response_dict['results']:
                #get order id
                new_order = order()
                new_order.fulltext = each_order
                new_order.order_id = each_order['id']
                new_order.status = each_order['status']
                #get payment nfo
                for each_payment in each_order['payments']:
                    new_payment = payment()
                    new_payment.payment_approvation_date = each_payment['date_approved']
                    new_payment.payment_status = each_payment['status']
                    new_payment.payment_id = each_payment['id']
                    new_payment.fulltext = each_payment
                    new_order.list_payments += [new_payment]
                #get items nfo
                for each_item in each_order['order_items']:
                    new_item = item()
                    new_item.fulltext = each_item
                    new_order.list_items += [new_item]
                #get shipment nfo
                new_shipment = shipment()
                new_shipment.shipment_id = each_order['shipping']['id']
                new_order.list_shipments += [new_shipment]

                #add to orders list
                self.client.orders_list += [new_order]
                
    def _getshippingnfo(self, shipment_id, this_shipment):
        extraparams = {
            }
        #add a header
        overrideHeaders = {
            "Authorization": "Bearer %s" %self.client.access_token
            }
        ending = str(shipment_id)
        
        response = setupAPIrequest(globals()['util']['meli_get_shipping_nfo'], extraparams, overrideHeaders, ending)
        response_dict = json.loads(response.text)
        this_shipment.fulltext = response_dict
        this_shipment.estimated_delivery_limit = response_dict['shipping_option']['estimated_delivery_limit']
        this_shipment.estimated_handling_limit = response_dict['shipping_option']['estimated_handling_limit']
        this_shipment.shipment_id = response_dict['id']
        this_shipment.shipment_substatus = response_dict['substatus']
        this_shipment.shipment_status = response_dict['status']
        this_shipment.tracking_number = response_dict['tracking_number']
        this_shipment.tracking_method = response_dict['tracking_method']

    def _getsellerreputation(self):
        extraparams = {
            }
        #add a header
        overrideHeaders = {
            "Authorization": "Bearer %s" %self.client.access_token
            }
        ending = str(self.client.user_id)
        response = setupAPIrequest(globals()['util']['meli_get_seller_reputation'], extraparams, overrideHeaders, ending)
        response_dict = json.loads(response.text)
        #only one reputation for each seller
        this_seller_reputation = reputation()
        this_seller_reputation.rawtext = response_dict
        this_seller_reputation.rawtext['result_date'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        this_seller_reputation.level_id = response_dict['seller_reputation']['level_id']
        this_seller_reputation.result_date = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        #the reputation_raw needs a column of date to track evolution of the reputation

        self.client.seller_reputation = this_seller_reputation
        
    def _remove_keys(self, dictionary, keys_to_remove):
        #to remove keys of a dictionary
        return {k: v for k, v in dictionary.items() if k not in keys_to_remove}
    
    def threadMainTask(self):
        #get the access token from meli
        self._getaccesstoken()

        #insert into newTokens list for insertion on nfo_meliTokenTable (to keep updated)
        globals()['list_response_newTokens'] += [{
            'client_id': self.client._client_id,
            'used_refresh_token': self.client.used_refresh_token,
            'refresh_time': datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
            'new_refesh_token': self.client.new_refresh_token
            }]

        #get orders from meli of the client
        self._getordersfromseller()

        #get reputation of the client
        self._getsellerreputation()
        
        #get shipping information of the order
        for each_order in self.client.orders_list:
            for each_shipment in each_order.list_shipments:
                self._getshippingnfo(each_shipment.shipment_id, each_shipment)

        #start data processing (T)
        #this is specific for asked table
        result_client_name = self.client._client_name
        result_client_id = self.client.client_id
        for each_order in self.client.orders_list:
            result_order_id = each_order.order_id
            for each_shipment in each_order.list_shipments:
                globals()['list_summaryTable_shipments'] += [{
                    'result_client_name' : result_client_name,
                    'result_client_id' : result_client_id,
                    'result_order_id' : result_order_id,
                    'result_shipment_id' : each_shipment.shipment_id,
                    'result_estimated_delivery_limit' : each_shipment.estimated_delivery_limit,
                    'result_estimated_handling_limit' : each_shipment.estimated_handling_limit,
                    'result_shipment_status' : each_shipment.shipment_status,
                    'result_shipment_shipment_substatus' : each_shipment.shipment_substatus,
                    'result_shipment_tracking_number' : each_shipment.tracking_number,
                    'result_shipment_tracking_method' : each_shipment.tracking_method
                }]
                #insert shipments info into full table
                globals()['list_rawTable_shipments'] += [each_shipment.fulltext]
                #x = pd.json_normalize(each_shipment.fulltext)
                #print (x)
            
            for each_payment in each_order.list_payments:
                globals()['list_summaryTable_payments'] += [{
                    'result_client_name' : result_client_name,
                    'result_client_id' : result_client_id,
                    'result_order_id' : result_order_id,
                    'result_payment_id' : each_payment.payment_id,
                    'result_payment_status' : each_payment.payment_status,
                    'result_payment_approvation_date' : each_payment.payment_approvation_date
                }]
                
            #this is to create the generic table (with all nfo)
            #print (each_order)
            globals()['list_rawTable_payments'] += each_order.fulltext['payments']
            
            #remove certain keys of each_order to make the rawTable_orders
            invalid = {"payments", "order_items","tags"}
            result_order = self._remove_keys(each_order.fulltext, invalid)
            globals()['list_rawTable_orders'] += [result_order]
        
        #place seller reputation on the global table
        #for summarized table
        globals()['list_summaryTable_reputation'] += [{
            'result_client_id' : result_client_id,
            'result_client_name' : result_client_name,
            'result_date' : self.client.seller_reputation.result_date,
            'result_level_id' : self.client.seller_reputation.level_id
        }]
        #for generic table
        globals()['list_rawTable_reputation'] += [self.client.seller_reputation.rawtext]
            
        